In [2]:
import requests
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import MultipleLocator

API_KEY = input("Por favor, introduzca su API Key aquí (sin comillas): ")

def extraccion_estacion(provincia, ciudad):

    url = "https://opendata.aemet.es/opendata/api/valores/climatologicos/inventarioestaciones/todasestaciones"
    response = requests.get(url, headers={"api_key": API_KEY})

    info = response.json()
    data_url = info["datos"]
    dataset = requests.get(data_url).json()  
    
    df = pd.DataFrame(dataset)
    mask = (df["provincia"] == provincia.upper()) & (df["nombre"] == ciudad.upper())
    df = df[mask]

    return df

extraccion_estacion("LAS PALMAS", "GRAN CANARIA AEROPUERTO")

,latitud,provincia,altitud,indicativo,nombre,indsinop,longitud
128,275504N,LAS PALMAS,24,C649I,GRAN CANARIA AEROPUERTO,60030,152343W


In [3]:

def extraccion_años(ano_ini, ano_fin, estacion):

    url = f"https://opendata.aemet.es/opendata/api/valores/climatologicos/mensualesanuales/datos/anioini/{ano_ini}/aniofin/{ano_fin}/estacion/{estacion}"

    response = requests.get(url, headers={"api_key": API_KEY})

    info = response.json()
    data_url = info["datos"]
    dataset = requests.get(data_url).json()  
 
    df = pd.DataFrame(dataset)

    col_interes = ["indicativo","fecha", "ta_max", "ta_min", "n_llu","w_racha", "hr","p_mes"]
    
    df_clean = df[col_interes].copy()

    df_clean.rename(columns={
        "ta_min": "Temperatura Minima", 
        "ta_max": "Temperatura Maxima",
        "n_llu": "Cantidad dias de lluvia",
        "w_racha": "velocidad del viento",
        "hr": "Humedad Relativa",
        "p_mes": "Precipitaciones mensuales"

    }, inplace=True)

    # convertir fecha
    df_clean["fecha"] = pd.to_datetime(df_clean["fecha"], errors="coerce")
    df_clean = df_clean.sort_values("fecha").reset_index(drop=True)

    # limpiar valores con paréntesis
    def limpiar_parentesis(valor):
        if isinstance(valor, str) and "(" in valor:
            return valor.split("(")[0].strip()
        return valor

    columnas_limpiar = ["Temperatura Maxima", "Temperatura Minima"]
    for col in columnas_limpiar:
        df_clean[col] = df_clean[col].apply(limpiar_parentesis)

   
    def limpiar_w_racha(valor):
        if isinstance(valor, str) and "/" in valor:
            return valor.split("/")[0].strip()
        return valor

    df_clean["velocidad del viento"] = df_clean["velocidad del viento"].apply(limpiar_w_racha)

    # convertir a numérico
    df_clean["Temperatura Maxima"] = pd.to_numeric(df_clean["Temperatura Maxima"], errors="coerce")
    df_clean["Temperatura Minima"] = pd.to_numeric(df_clean["Temperatura Minima"], errors="coerce")
    df_clean["Cantidad dias de lluvia"] = pd.to_numeric(df_clean["Cantidad dias de lluvia"], errors="coerce")
    df_clean["velocidad del viento"] = pd.to_numeric(df_clean["velocidad del viento"], errors="coerce")
    df_clean["Humedad Relativa"] = pd.to_numeric(df_clean["Humedad Relativa"], errors="coerce")
    df_clean["Precipitaciones mensuales"] = pd.to_numeric(df_clean["Precipitaciones mensuales"], errors="coerce")

    return df_clean

años = [(2005,2008),(2009,2011),(2012,2015),(2016,2019),(2020,2023),(2024,2024)]
estacion = "C649I"

resultados = [extraccion_años(ini, fin, estacion) for ini, fin in años]



for i, df in enumerate(resultados):
    
    df_clean = pd.concat(resultados,ignore_index=True)
    pd.set_option('display.max_rows', None)    
    pd.set_option('display.max_columns', None)  
    pd.set_option('display.width', None)        
    pd.set_option('display.max_colwidth', None) 
    df_clean

df_clean = df_clean.dropna(subset=["fecha"])
df_clean
df_clean = df_clean.interpolate().round(2)
df_clean

C:\Users\ignac\AppData\Local\Temp\ipykernel_18148\3731989770.py:77: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_clean = df_clean.interpolate().round(2)


,indicativo,fecha,Temperatura Maxima,Temperatura Minima,Cantidad dias de lluvia,velocidad del viento,Humedad Relativa,Precipitaciones mensuales
0,C649I,2005-01-01,22.60,12.00,6,2.00,56.00,15.0
1,C649I,2005-02-01,22.80,11.30,16,22.00,62.00,102.9
2,C649I,2005-03-01,27.80,12.10,5,21.00,66.00,16.7
3,C649I,2005-04-01,24.50,14.40,3,1.00,61.00,2.1
4,C649I,2005-05-01,26.30,16.50,4,3.00,62.00,0.0
5,C649I,2005-06-01,27.80,17.40,1,2.50,64.00,0.0
6,C649I,2005-07-01,33.70,19.00,0,2.00,59.00,0.0
7,C649I,2005-08-01,28.80,19.50,1,3.00,63.00,3.4
8,C649I,2005-09-01,37.70,20.80,3,2.00,62.00,6.6
9,C649I,2005-10-01,29.80,18.30,10,3.00,68.00,9.5


In [4]:
directory_path = r"..\Desktop\Proyecto AEMET\Data"
os.makedirs(directory_path, exist_ok=True)

df_clean.to_csv(r"..\Desktop\Proyecto AEMET\Data\las_palmas.csv", index=False)

In [5]:
 # valores medios
def valores_medios(Dataframe):
    medias_tramos = []
    for (ini, fin), df_clean in zip(años, resultados):
        media = df_clean.mean(numeric_only=True).round()
        media["años"] = f"{ini}-{fin}"
        medias_tramos.append(media)
    df_medias = pd.DataFrame(medias_tramos)
    return df_medias

df_valores_medios = valores_medios(df_clean)
df_valores_medios

,Temperatura Maxima,Temperatura Minima,Cantidad dias de lluvia,velocidad del viento,Humedad Relativa,Precipitaciones mensuales,años
0,29.0,15.0,8.0,31.0,61.0,29.0,2005-2008
1,30.0,16.0,4.0,31.0,63.0,17.0,2009-2011
2,29.0,16.0,9.0,23.0,64.0,17.0,2012-2015
3,29.0,16.0,9.0,22.0,64.0,17.0,2016-2019
4,30.0,16.0,10.0,25.0,64.0,17.0,2020-2023
5,30.0,17.0,9.0,16.0,64.0,13.0,2024-2024


In [6]:
 # valores maximos
def valores_maximos(Dataframe):
    medias_tramos = []
    for (ini, fin), df_clean in zip(años, resultados):
        media = df_clean.max(numeric_only=True).round()
        media["años"] = f"{ini}-{fin}"
        medias_tramos.append(media)
    df_medias = pd.DataFrame(medias_tramos)
    return df_medias

df_valores_maximos = valores_maximos(df_clean)
df_valores_maximos

,Temperatura Maxima,Temperatura Minima,Cantidad dias de lluvia,velocidad del viento,Humedad Relativa,Precipitaciones mensuales,años
0,39.0,21.0,66.0,99.0,68.0,308.0,2005-2008
1,39.0,21.0,34.0,99.0,69.0,175.0,2009-2011
2,37.0,21.0,75.0,99.0,72.0,163.0,2012-2015
3,38.0,22.0,74.0,99.0,72.0,158.0,2016-2019
4,36.0,22.0,70.0,99.0,72.0,207.0,2020-2023
5,35.0,21.0,59.0,99.0,71.0,84.0,2024-2024


In [7]:
 # valores minimos
def valores_minimos(Dataframe):
    medias_tramos = []
    for (ini, fin), df_clean in zip(años, resultados):
        media = df_clean.min(numeric_only=True).round()
        media["años"] = f"{ini}-{fin}"
        medias_tramos.append(media)
    df_medias = pd.DataFrame(medias_tramos)
    return df_medias

df_valores_minimos = valores_minimos(df_clean)
df_valores_minimos

,Temperatura Maxima,Temperatura Minima,Cantidad dias de lluvia,velocidad del viento,Humedad Relativa,Precipitaciones mensuales,años
0,21.0,11.0,0.0,1.0,51.0,0.0,2005-2008
1,22.0,10.0,0.0,1.0,58.0,0.0,2009-2011
2,21.0,10.0,0.0,1.0,58.0,0.0,2012-2015
3,22.0,10.0,0.0,1.0,54.0,0.0,2016-2019
4,24.0,11.0,0.0,1.0,54.0,0.0,2020-2023
5,27.0,13.0,0.0,1.0,59.0,0.0,2024-2024


In [8]:
df_valores_medios.to_csv("../Desktop/Proyecto AEMET/Data/las_palmas_medios.csv", index = False)
df_valores_maximos.to_csv("../Desktop/Proyecto AEMET/Data/las_palmas_maximos.csv" , index = False)
df_valores_minimos.to_csv("../Desktop/Proyecto AEMET/Data/las_palmas_minimos.csv" , index = False)